In [1]:
import yfinance as yf
import csv
import pandas as pd
import datetime

In [207]:
# GET LIST OF COMPANIES
# Get the symbol/ticker of the companies in the NASDAQ market from CSV previously downloaded from
# https://www.nasdaq.com/market-activity/stocks/screener
names = pd.read_csv('/home/jorgeluisgg/code/jorgeluisgg/project_awesome/raw_data/nasdaq_screener.csv')
print(names.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7004 entries, 0 to 7003
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      7003 non-null   object 
 1   Name        7004 non-null   object 
 2   Last Sale   7004 non-null   object 
 3   Net Change  7004 non-null   float64
 4   % Change    7004 non-null   object 
 5   Market Cap  6600 non-null   float64
 6   Country     6709 non-null   object 
 7   IPO Year    3948 non-null   float64
 8   Volume      7004 non-null   int64  
 9   Sector      6408 non-null   object 
 10  Industry    6407 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 602.0+ KB
None


In [237]:
# Create list with all tickers to iterate with the yfinance API
tickers_list = names[names.columns[0]].tolist()
tickers = [str(ticker) for ticker in tickers_list if "^" not in str(ticker)]
print(f'The number of companies available in the nasdaq list is: {len(tickers)}.')

The number of companies available in the nasdaq list is: 6622.


In [238]:
# Create list of yfinance objects
stocks = []
# Loop to fetch object for each ticker
for ticker in tickers:
    # Append the object to the list
    stocks.append(yf.Ticker(ticker))

print(f'The number of companies that were retrieved is: {len(stocks)}.')
# stocks[0:5],stocks[-5:]

The number of companies that were retrieved is: 6622.


In [228]:
# FLATTEN THE INFORMATION TO CREATE A DATAFRAME
# Create function to flatten information
def flatten_data(df,ticker):
    # Unstack the dataframe and reset the index
    df_ = df.unstack().reset_index()

    # Create the new column with row values containing the year and the feature names combined
    df_['Ticker'] = df_['level_0'].dt.year.astype(str) + ' ' + df_['level_1']

    # Drop unnecessary columns and transpose the column with names as the new head
    df_transposed = df_.drop(columns=['level_0', 'level_1']).set_index('Ticker').T

    # Add the Ticker column and set it as index
    df_transposed['Ticker'] = ticker
    df_transposed.set_index('Ticker', inplace=True)

    return df_transposed

In [301]:
# EXAMPLE OF THE FUNCTION WORKING
company = yf.Ticker("AAPL")
# flatten_data(company.cash_flow, company.ticker)
# stocks[42:44]
# stocks[42].balance_sheet
company.balance_sheet

,2023-09-30,2022-09-30,2021-09-30,2020-09-30,2019-09-30
Treasury Shares Number,0.0,NaN,NaN,NaN,NaN
Ordinary Shares Number,15550061000.0,15943425000.0,16426786000.0,16976763000.0,NaN
Share Issued,15550061000.0,15943425000.0,16426786000.0,16976763000.0,NaN
Net Debt,81123000000.0,96423000000.0,89779000000.0,74420000000.0,NaN
Total Debt,111088000000.0,120069000000.0,124719000000.0,112436000000.0,NaN
...,...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,61555000000.0,48304000000.0,62639000000.0,90943000000.0,NaN
Other Short Term Investments,31590000000.0,24658000000.0,27699000000.0,52927000000.0,NaN
Cash And Cash Equivalents,29965000000.0,23646000000.0,34940000000.0,38016000000.0,NaN
Cash Equivalents,1606000000.0,5100000000.0,17635000000.0,20243000000.0,NaN


In [314]:
data = []
for stock in stocks[42:44]:
    data.append(flatten_data(stock.balance_sheet,stock.ticker))

df_cf = pd.concat(data,axis = 0, ignore_index = False)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [312]:
# df_cf.to_csv(f'/home/jorgeluisgg/code/jorgeluisgg/project_awesome/raw_data/balance_sheets.csv')
df_cf

Ticker,2023 Treasury Shares Number,2023 Ordinary Shares Number,2023 Share Issued,2023 Net Debt,2023 Total Debt,2023 Tangible Book Value,2023 Invested Capital,2023 Working Capital,2023 Net Tangible Assets,2023 Common Stock Equity,...,2021 Total Assets,2021 Total Non Current Assets,2021 Other Non Current Assets,2021 Non Current Deferred Assets,2021 Current Assets,2021 Restricted Cash,2021 Prepaid Assets,2021 Cash Cash Equivalents And Short Term Investments,2021 Cash And Cash Equivalents,2021 Cash Financial
Ticker,,,,,,,,,,,,,,,,,,,,,
ACAB,0.0,8167391.0,8167391.0,NaN,160000.0,-8522477.0,-8362477.0,-5394928.0,-8522477.0,-8522477.0,...,175634.0,175634.0,NaN,175634.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# data = []
# for stock in stocks:
#     data.append(flatten_data(stock.income_stmt,stock.ticker))
# df_cf = pd.concat(data,axis = 0)

# df_cf.to_csv(f'/home/jorgeluisgg/code/jorgeluisgg/project_awesome/raw_data/income_statements.csv')

In [ ]:
# data = []
# for stock in stocks:
#     data.append(flatten_data(stock.cash_flow,stock.ticker))
# df_cf = pd.concat(data,axis = 0)

# df_cf.to_csv(f'/home/jorgeluisgg/code/jorgeluisgg/project_awesome/raw_data/cash_flows.csv')

In [297]:
import requests
import time

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import csv


# Get the list of NASDAQ companies
nasdaq_tickers = pd.read_csv('nasdaq_screener.csv')
#nasdaq_tickers= pd.read_csv('https://datahub.io/core/nasdaq-listings/r/nasdaq-listed-symbols.csv')
tickers_total = nasdaq_tickers['Symbol'].tolist()
tickers = tickers_total[0:100]

# Prepare the CSV file
header_written = False
csv_file = 'nasdaq_financial_data.csv'


# Helper function to get the stock price on a specific date
def get_stock_price(ticker, date, stock):
    stock_data = stock.history(start=date, end=date + timedelta(days=7))
    for i in range(7):
        try:
            adjusted_date = date + timedelta(days=i)
            stock_data_l = stock.history(start=adjusted_date, end=adjusted_date+timedelta(days=1))
            if not stock_data_l.empty:
                return stock_data_l['Close'].iloc[0]
        except Exception as e:
            print(f"get_stock_price : Error processing ticker {ticker}: {e}")



# Function that flattens the data of a dataframe concatenate the columns and put everything in a row
def flatten_data(df, ticker):

    df_u = df.unstack().reset_index()
    df_u['Ticker'] = df_u.apply(lambda row: str(row['level_0'].year) + ' ' + row['level_1'], axis=1)
    df_u.set_index(keys = 'Ticker')
    df_ordered = df_u.drop(columns = ['level_0','level_1'])
    df_transposed = df_ordered.set_index(keys='Ticker').T
    df_transposed['Ticker'] = ticker
    df_transposed.set_index(keys='Ticker',inplace = True)

    return df_transposed


#Compute the stock value ratio (item_stock%/sp500_stock%) in a year
def get_stock_value_ratio(date, ticker, stock, stock_sp):
    try:
        stock_data = get_stock_price(ticker, date, stock)
        if stock_data is not None:
            date_diff = date - timedelta(weeks=52)
            #change in s&p 500 price in %
            sp500_change = get_stock_price("^GSPC", date, stock_sp) / get_stock_price("^GSPC",date_diff, stock_sp)
            #change in stock price in %
            stock_change = get_stock_price(ticker, date, stock)/ get_stock_price(ticker,date_diff, stock)
            #ratio of change_stock / change_s&p500
            ratio = stock_change / sp500_change
            if ratio is not None:
                return ratio
    except Exception as e:
        print(f"get_stock_value_ratio : Error processing ticker {ticker}: {e}")

#function to get the stock value corresponding to the date date

def get_stock_value(df, ticker, stock, stock_sp) :
    lst_stock_value = []
    lst_stock_ratio = []
    for date in df.columns :
        date_str = date.strftime('%Y-%m-%d')
        lst_stock_value.append(get_stock_price(ticker, date, stock))
        lst_stock_ratio.append(get_stock_value_ratio(date, ticker, stock, stock_sp))

    df.loc['Stock value'] = lst_stock_value
    df.loc['Ratio ticker sp500'] = lst_stock_ratio
    return df

def create_df(tickers):
    stock_sp = yf.Ticker('^GSPC')
    df_complete = pd.DataFrame()
    for ticker in tickers:
        df_row = pd.DataFrame()
        try:
            # Get financial statements for the ticker ticker
            stock = yf.Ticker(ticker)
            balance_sheet = stock.balance_sheet
            income_statement = stock.financials
            cashflow_statement = stock.cashflow
            #stock_history = stock.history

            # add the stock value and flatten the dataframe
            df_bs = get_stock_value(balance_sheet,ticker, stock, stock_sp)
            df_bs_f = flatten_data(df_bs,ticker)
            df_is_f = flatten_data(income_statement,ticker)
            df_cs_f = flatten_data(cashflow_statement,ticker)
            #concatenate
            df_row = pd.concat([df_bs_f,df_is_f,df_cs_f],axis=1)
            df_complete = pd.concat([df_complete, df_row])

        except Exception as e:
            print(f"create_df : Error processing ticker {ticker}: {e}")
    return df_complete